# Step 1

## 01. Import Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

## 02. Set Paths and Import Data

In [2]:
path = r'C:\Users\danve\Documents\09.2022_Instacart_Basket_Analysis'

In [3]:
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [4]:
dfords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

## 03. Make a New Test Data Frame

In [5]:
# Create a dataframe
df_test = pd.DataFrame()

In [6]:
# Create a mixed type column
df_test['mix'] = ['a', 'b', 1, True]

In [7]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [8]:
## Check for mixed types
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


## 04. Find Missing Values

In [9]:
## Check for nulls
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [11]:
## Create a subset for the null values
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [12]:
## Missing product names
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


## 05. Addressing Missing Values

In [13]:
## Compare missing rows versus null subset
df_prods.shape

(49693, 5)

In [17]:
## Create a new dataframe
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [18]:
df_prods_clean.shape

(49677, 5)

## 06. Duplicates

In [19]:
## Look for full duplicates

In [20]:
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [21]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


## 07. Addressing Duplicates

In [22]:
## Check number of rows
df_prods_clean.shape

(49677, 5)

In [23]:
## Anything about the data that looks off or should be investigated further. 
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [24]:
df_prods_clean_no_dups.shape

(49672, 5)

# Step 2

In [25]:
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


In [26]:
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


### I'm not sure what is going on with the max price value 99999 -- this looks like an unusual value, probably a placeholder?

# Step 3

In [28]:
## Check for mixed types in orders -- the exercise wanted the orders dataframe sets as "dfords" for some reason.
for col in dfords.columns.tolist():
  weird = (dfords[[col]].applymap(type) != dfords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (dfords[weird]) > 0:
    print (col)

###  This did not return any results, so no mixed dtypes

# Step 4

### Q. If you find mixed-type data, fix it. The column in question should contain observations of a single data type.

### A. My check did not yield any mixed types. 

# Step 5

In [30]:
## Run a check for missing values in your df_ords dataframe.
dfords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
user_order_number              0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

### There seems to be missing data in the days_since_prior_order, but if you look at the raw data the missing values for days_since_prior_order are probably due to the fact that the very first order that a customer makes will not have a 'prior order' date to calculate a value.
### To addess this we could: 1) create a flag variable to callout the missing value; 2) impute the missing value; 3) remove the missing value

# Step 6

In [33]:
## Address the missing values using an appropriate method. 
dfords.describe()

,Unnamed: 0,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


### If we look at the number of user_ids, using the max number of 2.062090e+05 -- which seems an unnecessarily complicated way of saying 206,209 -- we know that there are a total of 206,209 users who have placed orders with instacart, which also means that each of the individual 206,206 users had to place their first/1st order for which there would be no "prior order" and hence no 'days_since_prior_order' value for 206,209 orders which corresponds with the number of null/missing 206209 values for 'days_since_prior_order' in Step 5. 

### Since the data for these first time orders are useful, our best apprpoach would be to create a new variable that acts like a flag based on the missing value

In [36]:
## Create a new column to indicate first order
dfords['first_order'] = np.where(dfords['days_since_prior_order'].isnull(), 'yes', 'no')

In [37]:
## I was not sure what to do for this, so I looked up the code above from the internet:/
dfords

,Unnamed: 0,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order
0,0,2539329,1,1,2,8,NaN,yes
1,1,2398795,1,2,3,7,15.0,no
2,2,473747,1,3,3,12,21.0,no
3,3,2254736,1,4,4,7,29.0,no
4,4,431534,1,5,4,15,28.0,no
...,...,...,...,...,...,...,...,...
3421078,3421078,2266710,206209,10,5,18,29.0,no
3421079,3421079,1854736,206209,11,4,10,30.0,no
3421080,3421080,626363,206209,12,1,12,18.0,no
3421081,3421081,2977660,206209,13,1,12,7.0,no


In [38]:
## 'Clean' dataframe
dfords_clean = dfords

In [40]:
dfords_clean

,Unnamed: 0,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order
0,0,2539329,1,1,2,8,NaN,yes
1,1,2398795,1,2,3,7,15.0,no
2,2,473747,1,3,3,12,21.0,no
3,3,2254736,1,4,4,7,29.0,no
4,4,431534,1,5,4,15,28.0,no
...,...,...,...,...,...,...,...,...
3421078,3421078,2266710,206209,10,5,18,29.0,no
3421079,3421079,1854736,206209,11,4,10,30.0,no
3421080,3421080,626363,206209,12,1,12,18.0,no
3421081,3421081,2977660,206209,13,1,12,7.0,no


# Step 7

In [41]:
## Run a check for duplicate values in your df_ords data.
dfords_dups = dfords_clean[dfords_clean.duplicated()]

In [42]:
dfords_dups

,Unnamed: 0,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order


### I don't think there are any full duplicate values to deal with

# Step 8

### Q. Address the duplicates using an appropriate method.

### A. I don't I have any duplicates, but I would remove them

In [43]:
dfords_clean.shape

(3421083, 8)

In [45]:
dfords_clean_no_dups = dfords_clean.drop_duplicates()

In [46]:
dfords_clean_no_dups.shape

(3421083, 8)

# Step 9

In [47]:
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'))

In [48]:
dfords_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_checked.csv'))